### --- 0. Πειραματικό Πλαίσιο και Μεθοδολογία Ανάλυσης ---

Το παρόν R script υλοποιεί τη στατιστική μοντελοποίηση που ακολουθεί την εξαγωγή ακουστικών χαρακτηριστικών. Σκοπός του είναι να διερευνήσει συστηματικά ποια χαρακτηριστικά μπορούν να προβλέψουν σημαντικές κλινικές μεταβλητές.

**Πηγή Δεδομένων και Προσέγγιση:**
Το script χρησιμοποιεί ένα ενιαίο, περιεκτικό αρχείο εισόδου (`Patient_Sorted_Updated_Final_Extracted_Features.xlsx`).

**Είναι κρίσιμης σημασίας ότι αυτό το αρχείο περιέχει συνδυαστικά τα δεδομένα και από τις δύο φαρμακευτικές καταστάσεις, 'ON' (υπό αγωγή) και 'OFF' (χωρίς αγωγή)**, για το σύνολο των ασθενών.

Τα στατιστικά μοντέλα που κατασκευάζονται σε αυτό το script προσαρμόζονται σε αυτό το μικτό σύνολο δεδομένων. Για κάθε κλινικό αποτέλεσμα και φωνητικό task, η ανάλυση **δεν διαχωρίζει τις καταστάσεις 'ON' και 'OFF'**.

**Ερμηνεία Αποτελεσμάτων:**
Κατά συνέπεια, τα αποτελέσματα αυτών των μοντέλων (π.χ., η στατιστική σημαντικότητα των predictors) πρέπει να ερμηνεύονται ως η **συνολική προβλεπτική ικανότητα** των ακουστικών χαρακτηριστικών, συνδυάζοντας την επίδραση και από τις δύο φαρμακευτικές καταστάσεις.

In [ ]:
# Εγκατάσταση απαραίτητων πακέτων αν δεν υπάρχουν ήδη
install.packages("writexl")
install.packages("lmerTest")

In [ ]:
# --- 1. Φόρτωση Απαραίτητων Βιβλιοθηκών ---
library(writexl)
library(readxl)
library(lmerTest)
library(dplyr)

# --- 2. Φόρτωση και Προετοιμασία Δεδομένων ---
# Ορισμός της διαδρομής του αρχείου για το κύριο σύνολο δεδομένων.
file_path <- "Patient_Sorted_Updated_Final_Extracted_Features.xlsx"
# Ανάγνωση των δεδομένων από το καθορισμένο αρχείο Excel σε ένα data frame με το όνομα 'data'.
data <- read_excel(file_path)

# Ορισμός βοηθητικής συνάρτησης για το φιλτράρισμα του κύριου data frame.
filter_data <- function(data, file_names = NULL) {
  if (!is.null(file_names)) {
    data <- data[data$file_name %in% file_names, ]
  }
  return(data)
}

# Ορισμός των εξαρτημένων μεταβλητών (κλινικών αποτελεσμάτων) που θα προβλεφθούν.
dependent_variables <- c("H_Y_binary", "Fluctuations", "Dyskinesia", "UPDRS_III_dif_class", "UPDRS_III_dif_per")

# Ορισμός των φωνητικών tasks (εργασιών).
tasks <- list(
  task1 = c("a1.wav", "a2.wav", "a3.wav", "z1.wav", "z2.wav", "z3.wav", "s1.wav", "s2.wav", "s3.wav"),
  task2 = c("sakis.wav", "say.wav", "school.wav", "wind.wav", "yesno.wav", "lentils.wav", "mom1.wav", "mom2.wav", "illness.wav", "earth.wav", "ball1.wav", "ball2.wav"),
  task3 = c("rainbow_passage.wav"),
  task4 = c("oe.wav", "pa.wav", "pataka.wav", "ta.wav", "ka.wav", "kala.wav", "aui.wav"),
  task5 = c("f_autoind.wav", "f_bat.wav", "f_bicycle.wav", "f_cinema.wav", "f_cockroachkiller.wav", "f_mailman.wav", "f_pastrychef.wav", "f_radio.wav", "f_ring.wav", "f_rose.wav", "f_safe.wav", "f_snail.wav", "f_spaceship.wav", "f_telecom.wav", "s_autoind.wav", "s_bat.wav", "s_bicycle.wav", "s_cinema.wav", "s_cockroachkiller.wav", "s_mailman.wav", "s_pastrychef.wav", "s_radio.wav", "s_ring.wav", "s_rose.wav", "s_safe.wav", "s_snail.wav", "s_spaceship.wav", "s_telecom.wav"),
  task6 = c("counting.wav"),
  task7 = c("breathing.wav"),
  task8 = c("coughing.wav")
)

# --- 3. Αρχικοποίηση Data Frames για την Αποθήκευση Αποτελεσμάτων ---
task_summary <- data.frame(
  Task = names(tasks),
  Exercises = sapply(tasks, function(x) paste(x, collapse = ", "))
)
final_results <- data.frame()

# --- 4. Κύριος Βρόχος Ανάλυσης ---
for (dep_var in dependent_variables) {
  cat("Εκτέλεση μοντέλων για την εξαρτημένη μεταβλητή:", dep_var, "\n")
  
  # Προσθήκη γραμμής-κεφαλίδας για σαφήνεια στο Excel
  final_results <- bind_rows(final_results, data.frame(
    Task = paste("--------Ανάλυση για", dep_var, "--------"), 
    Predictor = NA, 
    Estimate = NA, 
    Std.Error = NA, 
    t.value = NA, 
    p.value = NA
  ))
  
  for (task_name in names(tasks)) {
    file_names <- tasks[[task_name]]
    cat("Εκτέλεση μοντέλου για το task", task_name, "\n")
    
    # === 4α. ΜΟΝΤΕΛΟ ΓΙΑ ΟΛΟΚΛΗΡΟ ΤΟ TASK ===
    filtered_data <- filter_data(data, file_names = file_names)
    
    if (nrow(filtered_data) == 0) {
      cat("Δεν υπάρχουν δεδομένα για το task", task_name, "\n")
      next
    }
    
    # Προεπεξεργασία της εξαρτημένης μεταβλητής
    if (dep_var == "H_Y_binary") {
      filtered_data$H_Y_binary <- ifelse(filtered_data$`H&Y` < 3, 0, 1)
    } else if (dep_var == "Fluctuations") {
      filtered_data$Fluctuations <- ifelse(filtered_data$`Fluctuations` == "Yes", 1, 0)
    } else if (dep_var == "Dyskinesia") {
      filtered_data$Dyskinesia <- ifelse(filtered_data$`Dyskinesia` == "Yes", 1, 0)
    } else if (dep_var == "UPDRS_III_dif_class") {
      filtered_data$UPDRS_III_dif_class <- as.numeric(factor(filtered_data$UPDRS_III_dif_class, levels = c("Poor", "Suboptimal", "Excellent")))
    } else if (dep_var == "UPDRS_III_dif_per") {
      filtered_data$UPDRS_III_dif_per <- as.numeric(filtered_data$UPDRS_III_dif_per)
    }
    
    # Επιλογή των χαρακτηριστικών (στήλες 24 έως 111)
    features <- filtered_data[, 24:111]
    colnames(features) <- make.names(colnames(features))
    predictors <- paste(colnames(features), collapse = " + ")
    
    # Κατασκευή του τύπου για το LMM με random effect το file_name
    formula <- as.formula(paste(dep_var, "~", predictors, "+ (1 | file_name)"))
    
    model <- tryCatch(
      lmer(formula, data = filtered_data),
      error = function(e) {
        cat("Σφάλμα στην προσαρμογή του μοντέλου για το task", task_name, ":", e$message, "\n")
        return(NULL)
      }
    )
    
    if (!is.null(model)) {
      model_summary <- summary(model)
      fixed_effects <- model_summary$coefficients
      fixed_results_df <- data.frame(
        Task = paste(task_name, "(Όλες οι Ασκήσεις)"), 
        Predictor = rownames(fixed_effects),
        Estimate = fixed_effects[, "Estimate"],
        Std.Error = fixed_effects[, "Std. Error"],
        t.value = fixed_effects[, "t value"],
        p.value = fixed_effects[, "Pr(>|t|)"]
      )
      final_results <- bind_rows(final_results, fixed_results_df)
      final_results <- bind_rows(final_results, data.frame(Task=NA, Predictor=NA, Estimate=NA, Std.Error=NA, t.value=NA, p.value=NA))
    }
    
    # === 4β. ΜΟΝΤΕΛΟ ΓΙΑ ΚΑΘΕ ΜΕΜΟΝΩΜΕΝΗ ΑΣΚΗΣΗ ===
    for (file_name in file_names) {
      cat("Εκτέλεση μοντέλου για τη μεμονωμένη άσκηση:", file_name, "\n")
      filtered_data_single <- filter_data(data, file_names = file_name)
      
      if (nrow(filtered_data_single) == 0) {
        next
      }
      
      # Προεπεξεργασία της εξαρτημένης μεταβλητής (επανάληψη για το υποσύνολο)
      if (dep_var == "H_Y_binary") {
        filtered_data_single$H_Y_binary <- ifelse(filtered_data_single$`H&Y` < 3, 0, 1)
      } else if (dep_var == "Fluctuations") {
        filtered_data_single$Fluctuations <- ifelse(filtered_data_single$`Fluctuations` == "Yes", 1, 0)
      } else if (dep_var == "Dyskinesia") {
        filtered_data_single$Dyskinesia <- ifelse(filtered_data_single$`Dyskinesia` == "Yes", 1, 0)
      } else if (dep_var == "UPDRS_III_dif_class") {
        filtered_data_single$UPDRS_III_dif_class <- as.numeric(factor(filtered_data_single$UPDRS_III_dif_class, levels = c("Poor", "Suboptimal", "Excellent")))
      } else if (dep_var == "UPDRS_III_dif_per") {
        filtered_data_single$UPDRS_III_dif_per <- as.numeric(filtered_data_single$UPDRS_III_dif_per)
      }
      
      features_single <- filtered_data_single[, 24:111]
      colnames(features_single) <- make.names(colnames(features_single))
      predictors_single <- paste(colnames(features_single), collapse = " + ")
      
      # Για μεμονωμένη άσκηση, χρησιμοποιούμε απλό Γραμμικό Μοντέλο (LM)
      formula <- as.formula(paste(dep_var, "~", predictors_single))
      
      model_lm <- tryCatch(
        lm(formula, data = filtered_data_single),
        error = function(e) {
          cat("Σφάλμα στην προσαρμογή του LM για την άσκηση", file_name, ":", e$message, "\n")
          return(NULL)
        }
      )
      
      if (!is.null(model_lm)) {
        model_summary_lm <- summary(model_lm)
        effects_lm <- coef(model_summary_lm)
        results_df_lm <- data.frame(
          Task = file_name,
          Predictor = rownames(effects_lm),
          Estimate = effects_lm[, "Estimate"],
          Std.Error = effects_lm[, "Std. Error"],
          t.value = effects_lm[, "t value"],
          p.value = effects_lm[, "Pr(>|t|)"]
        )
        final_results <- bind_rows(final_results, results_df_lm)
        final_results <- bind_rows(final_results, data.frame(Task=NA, Predictor=NA, Estimate=NA, Std.Error=NA, t.value=NA, p.value=NA))
      }
    }
  }
}

# --- 5. Αποθήκευση Τελικών Αποτελεσμάτων ---
output_file_path <- "summary_all_models_detailed.xlsx"
write_xlsx(list("Task Summary" = task_summary, "Model Results" = final_results), output_file_path)

cat("\n\n*** Η ανάλυση ολοκληρώθηκε. Τα αποτελέσματα αποθηκεύτηκαν στο:", output_file_path, "***\n")